<a href="https://colab.research.google.com/github/elbergyn/python/blob/master/NASA_Julho_95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version


In [0]:
! pip install pyspark

In [0]:
from pyspark import SparkContext
from operator import add

In [0]:
sc = SparkContext()

In [0]:
!git clone https://github.com/leonardoamorim/aulapython.git

In [0]:
! gunzip aulapython/NASA_access_log_Jul95.gz
! gunzip aulapython/NASA_access_log_Aug95.gz

In [0]:
arquivo = sc.textFile("aulapython/NASA_access_log_Jul95")#carregando arquivo 

In [0]:
def count_hosts_distintos(dados):
  return dados.map(lambda line: line.split(' ')[0]).distinct().count()

In [0]:
hosts_dintintos_julho = count_hosts_distintos(arquivo)

In [0]:
# Funcao para verificar se em uma linha existe um codigo http igual a 404
def codigo404(linha):
    try:
        codigohttp = linha.split(' ')[-2]
        print(codigohttp)
        if codigohttp == '404':
            return True
    except:
        pass
    return False

In [0]:
erros404_julho = arquivo.filter(codigo404)

In [0]:
def top5_erro404(dados):
  hosts = dados.map(lambda lin: lin.split()[6])
  counts = hosts.map(lambda lin: (lin, 1)).reduceByKey(add)#somando o número de vezes chamado
  top5 = counts.sortBy(lambda par: -par[1]).take(5) #ordenar decrescente pela quatidade e retornar os 5 primeiros
  
  retorno = ''
  for host, erro in top5:
    retorno += '\n' + host + ': ' + str(erro)

  return retorno;

In [0]:
top5_erros404_julho = top5_erro404(erros404_julho)

In [0]:
def erros404_dia(dados):
  dias = dados.map(lambda lin: lin.split('[')[1].split(':')[0])
  counts = dias.map(lambda dia: (dia, 1)).reduceByKey(add)
  return counts.sortBy(lambda data: data[0]).collect()

In [0]:
erro404_dia_julho = erros404_dia(erros404_julho)

In [0]:
def valor_bytes(linha):
    try:
        tamanho = int(linha.split()[-1])
        return tamanho        
    except:
        return 0

In [0]:
total_bytes = arquivo.map(lambda lin: valor_bytes(lin)).reduce(lambda a, b: a + b)

In [0]:
print('Números de hosts distintos em Julho: ', hosts_dintintos_julho)

In [0]:
print('Total de erros 404 em Julho: ',erros404_julho.count())

In [39]:
print('Top5 urls com erro 404 em julho: ', top5_erros404_julho)

Top5 urls com erro 404 em julho:  
/pub/winvn/readme.txt: 667
/pub/winvn/release.txt: 547
/history/apollo/apollo-13.html: 286
/history/apollo/a-001/a-001-patch-small.gif: 230
/shuttle/resources/orbiters/atlantis.gif: 230


In [0]:
print('Erros 404 por dia em julho')
for dia, total in erro404_dia_julho:
  print(dia, total)

In [0]:
print('Total de bytes recebidos em Julho: ', total_bytes)